In [1]:
import numpy as np
import pandas as pd

In [2]:
data = pd.read_csv('banknotes.txt', sep='\t')

In [10]:
data.head()

,X1,X2,X3,X4,X5,X6,real
0,214.8,131.0,131.1,9.0,9.7,141.0,1
1,214.6,129.7,129.7,8.1,9.5,141.7,1
2,214.8,129.7,129.7,8.7,9.6,142.2,1
3,214.8,129.7,129.6,7.5,10.4,142.0,1
4,215.0,129.6,129.7,10.4,7.7,141.8,1


In [11]:
X = data.drop(['real'], axis=1)
y = data.real

In [6]:
from sklearn import model_selection, linear_model

In [99]:
train_data, test_data, train_labels, test_labels = model_selection.train_test_split(X, y, test_size=0.25, random_state=1)

In [100]:
model1 = linear_model.LogisticRegression(random_state=1)
model2 = linear_model.LogisticRegression(random_state=1)

In [101]:
model1.fit(train_data[['X1', 'X2', 'X3']], train_labels)
model2.fit(train_data[['X4', 'X5', 'X6']], train_labels)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=1, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [102]:
def p(predicts, labels):
    return np.sum(np.abs(predicts - labels))/len(labels)

## Z-критерий для разности долей (связанные выборки)

  $X_1$ \ $X_2$ | 1| 0 | $\sum$
  ------------- | -------------|
  1  | e | f | e + f
  0  | g | h | g + h
  $\sum$ | e + g| f + h | n  
  
$$ \hat{p}_1 = \frac{e + f}{n}$$

$$ \hat{p}_2 = \frac{e + g}{n}$$

$$ \hat{p}_1 - \hat{p}_2 = \frac{f - g}{n}$$


$$\text{Доверительный интервал для }p_1 - p_2\colon \;\;  \frac{f - g}{n} \pm z_{1-\frac{\alpha}{2}}\sqrt{\frac{f + g}{n^2} - \frac{(f - g)^2}{n^3}}$$

$$Z-статистика: Z({X_1, X_2}) = \frac{f - g}{\sqrt{f + g - \frac{(f-g)^2}{n}}}$$

In [103]:
import scipy
from statsmodels.stats.weightstats import *
from statsmodels.stats.proportion import proportion_confint

In [104]:
def proportions_diff_confint_rel(sample1, sample2, alpha = 0.05):
    z = scipy.stats.norm.ppf(1 - alpha / 2.)
    sample = list(zip(sample1, sample2))
    n = len(sample)
        
    f = sum([1 if (x[0] == 1 and x[1] == 0) else 0 for x in sample])
    g = sum([1 if (x[0] == 0 and x[1] == 1) else 0 for x in sample])
    
    left_boundary = float(f - g) / n  - z * np.sqrt(float((f + g)) / n**2 - float((f - g)**2) / n**3)
    right_boundary = float(f - g) / n  + z * np.sqrt(float((f + g)) / n**2 - float((f - g)**2) / n**3)
    return (left_boundary, right_boundary)

In [105]:
def proportions_diff_z_stat_rel(sample1, sample2):
    sample = list(zip(sample1, sample2))
    n = len(sample)
    
    f = sum([1 if (x[0] == True and x[1] == False) else 0 for x in sample])
    g = sum([1 if (x[0] == False and x[1] == True) else 0 for x in sample])
    
    return float(f - g) / np.sqrt(f + g - float((f - g)**2) / n )

In [106]:
def proportions_diff_z_test(z_stat, alternative = 'two-sided'):
    if alternative not in ('two-sided', 'less', 'greater'):
        raise ValueError("alternative not recognized\n"
                         "should be 'two-sided', 'less' or 'greater'")
    
    if alternative == 'two-sided':
        return 2 * (1 - scipy.stats.norm.cdf(np.abs(z_stat)))
    
    if alternative == 'less':
        return scipy.stats.norm.cdf(z_stat)

    if alternative == 'greater':
        return 1 - scipy.stats.norm.cdf(z_stat)

In [107]:
print("p-value: %f" % proportions_diff_z_test(proportions_diff_z_stat_rel(model1.predict(test_data[['X1', 'X2', 'X3']]) == test_labels, 
                                                                          model2.predict(test_data[['X4', 'X5', 'X6']]) == test_labels)))

p-value: 0.003297


In [111]:
print ("95%% confidence interval for a difference between proportions: [%f, %f]" \
      % proportions_diff_confint_rel(np.abs(model1.predict(test_data[['X1', 'X2', 'X3']]) - test_labels), 
                                     np.abs(model2.predict(test_data[['X4', 'X5', 'X6']]) - test_labels)))

95% confidence interval for a difference between proportions: [0.059945, 0.300055]
